In [ ]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
from sklearn import linear_model        #表示，可以调用sklearn中的linear_model模块进行线性回归。
import numpy as np
er_temp_df = pd.read_csv('.\\raw\\dataset-S4.CSV').iloc[:,1:]
dr_temp_df = pd.read_csv('.\\raw\\dataset-S5.CSV').iloc[:,[1,2,3,5,6,7]]
er_moisture_df = pd.read_csv('.\\raw\\fungal_biogeography\\fungi_data\\Fungi_moisture_curves.csv')
traites_df = pd.read_csv('.\\raw\\Fungal_trait_data.csv')# .iloc[:,[1,2,3,5,6,7]]

# Data Preprocessing

In [ ]:
er_temp_tensor = torch.tensor(er_temp_df.values)
dr_temp_tensor = torch.tensor(dr_temp_df.values)

In [ ]:
er_temp = pd.DataFrame(columns=['slope', 'intercept'])
for i in range(34):
    model = linear_model.LinearRegression()
    model.fit(np.array([[10],[16],[22]]), torch.log(er_temp_tensor[i, 0:3]))
    er_temp = er_temp.append({'slope':model.coef_[0], 'intercept':model.intercept_}, ignore_index=True)

er_temp.insert(2, 'range', torch.mean(er_temp_tensor[:,3:], dim=1).tolist())
er_temp.to_csv('.\\er_temp.csv', index_label=False)

In [ ]:
traites_df.iloc[:,1].to_csv('.\\traits_map.csv')

# Data Analysis

### Relationship Between Extension Rate and Temperature

In [ ]:
for i in range(34):
    plt.plot([10,16,22], er_temp_tensor[i,0:3])
plt.yscale('log')
plt.xlabel('temperature')
plt.ylabel('log (extension rate)')
plt.show()

In [ ]:
for i in range(34):
    plt.plot([10,16,22], er_temp_tensor[i,3:])
plt.xlabel('temperature')
plt.ylabel('delta extension rate)')
plt.show()

In [ ]:
for i in range(3):
    x = er_temp_tensor[:,i]
    y = dr_temp_tensor[:,i]
    X = torch.cat([x.reshape(34,1), x.reshape(34,1)**0.5], dim=1)
    model = linear_model.LinearRegression()
    model.fit(X,y)
    linspace = np.linspace(0,torch.max(x).item())
    polyspace = np.transpose([linspace, linspace**0.5])
    prediction = model.predict(polyspace)
    plt.scatter(x, y)
    plt.plot(linspace, prediction)
    plt.xlabel('extensition rate')
    plt.ylabel('decomposition rate')
plt.show()


In [ ]:
from torch import nn
class FungusMoisture(nn.Module):
    def __init__(self):
        super(FungusMoisture, self).__init__()
        layers = [nn.Linear(1, 32), nn.ELU(),nn.Linear(32, 256),nn.ELU(), nn.Linear(256, 1024),nn.ELU(),
                  nn.Linear(1024, 1024),nn.ELU(),nn.Linear(1024, 256), nn.ELU(),nn.Linear(256, 32),nn.ELU(),
                  nn.Linear(32, 1),nn.ELU()]
        self.learner = nn.Sequential(*layers)

    def forward(self, x):
        return self.learner(x)

    def save(self, name):
        torch.save(self.state_dict(), f'.\\er_wp_map\\{name}.txt')

    def load(self, name):
        self.load_state_dict(torch.load(f'.\\er_wp_map\\{name}.txt'))
        self.eval()

_traits_map_df = pd.read_csv('.\\traits_map.csv')
traits_map = dict(zip(_traits_map_df.iloc[:, 0].values, _traits_map_df.iloc[:, 1].values))



def plot(num_showing = 0):
    global traits_map, er_moisture_df
    model = FungusMoisture().cuda()
    model.load(num_showing)
    trait = traits_map[num_showing]
    groups = er_moisture_df.groupby('species').groups
    idxes = groups[trait]
    
    test_x = torch.linspace(-5,0,100, device='cuda:0').reshape(100,1,1)
    test_y = model(test_x)
    # plt.scatter(er_moisture_df.iloc[idxes,1], er_moisture_df.iloc[idxes,2])
    plt.plot(test_x.cpu().reshape(100), test_y.detach().cpu().reshape(100))
    

for i in range(34):
    plot(i)
plt.xlabel('Matric Water Potential')
plt.ylabel('Extension Rate')
plt.show()

In [ ]:

def scatter(num_showing = 0):
    global traits_map, er_moisture_df
    trait = traits_map[num_showing]
    groups = er_moisture_df.groupby('species').groups
    idxes = groups[trait]
    plt.scatter(er_moisture_df.iloc[idxes,1], er_moisture_df.iloc[idxes,2])
    
for i in range(34):
    scatter(i)
plt.xlabel('Matric Water Potential')
plt.ylabel('Extension Rate')
plt.show()